<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml0922/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml0922/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/tibaml0922/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f52b6debd50>)

In [2]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [3]:
import numpy as np
# y: 0, 1, 2...
# Series.replace({"李白":0, ...})
# Series.unique()/Series.value_counts()
artists = train_df["作者"].unique()
# artist2idx = {}
# for i, a in enumerate(artists):
#    artist2idx[a] = i
artist2idx = {a:i for i, a in enumerate(artists)}
idx2artist = {i:a for i, a in enumerate(artists)}
y_train = np.array(train_df["作者"].replace(artist2idx))
y_test = np.array(test_df["作者"].replace(artist2idx))
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
import jieba
# 因為古詩, 我才沒載入大辭典
# Series.apply(函式名字)
def poemcut(p):
    return " ".join(jieba.cut(p))
x_train_cut = train_df["內容"].apply(poemcut)
x_test_cut = test_df["內容"].apply(poemcut)
x_train_cut

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.987 seconds.
Prefix dict has been built successfully.


0       平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 \r\n 暝 色入 高樓 ， 有人 樓上 ...
1       青天 有 月 來 幾時 ， 我今 停杯 一問 之 ： 人攀 明月 不可 得 ， 月行 卻 與...
2       燕草 如碧絲 ， 秦桑低 綠枝 。 當君 懷歸日 ， 是 妾 斷腸時 。 春風 不 相識 ，...
3       誰 家玉笛 暗飛聲 ， 散 入春 風滿 洛城 。 此 夜曲 中聞折 柳 ， 何人 不起 故園情 。
4       西上 蓮 花山 ， 迢迢 見 明星 。 ( 西 上   一作 ： 西 嶽 ) 素 手把 芙蓉...
                              ...                        
2726    零落 桐葉雨 ， 蕭條 槿 花風 。 悠悠 早 秋意 ， 生此 幽閒 中 。 況 與 故人 ...
2727    離離 暑雲散 ， 嫋嫋 涼風起 。 池上 秋 又 來 ， 荷花 半成 子 。 朱顏易銷 歇 ...
2728    涼風 冷露 蕭索天 ， 黃 蒿 紫菊荒 涼田 。 繞 塚 秋花少 顏色 ， 細蟲 小蝶飛 翻...
2729    往歲 曾 為 西邑吏 ， 慣 從 駱口 到 南秦 。 \r\n 三時 雲冷多 飛雪 ， 二月...
2730    笛 竹出 蘄 春 ， 霜刀 劈翠筠 。 織成 雙鎖 簟 ， 寄 與 獨眠 人 。 卷作筒 中...
Name: 內容, Length: 2731, dtype: object

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train_cut)
# !!! 千萬不要再做一次fit, fit:找出欄位有多少個
x_test_count = vec.transform(x_test_cut)

In [6]:
len(vec.vocabulary_)
# 出現KeyError是對的
# vec.vocabulary_["。"]
# vec.vocabulary_["\r\n"]

52294

In [7]:
# print(x_train_count)
x_train_count

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [40]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [22]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8333333333333334

In [23]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat, 
             index=[a + "(真)" for a in artists],
             columns=[a + "(預測)" for a in artists])

,李白(預測),杜甫(預測),白居易(預測)
李白(真),8,1,1
杜甫(真),0,9,1
白居易(真),1,1,8


In [42]:
testp = input("請輸入要預測的詩:")
testpcut = [poemcut(testp)]
testpcount = vec.transform(testpcut)
proba = clf.predict_proba(testpcount)[0]
for a, p in zip(artists, proba):
    print(a, "的機率:", p)
ans = clf.predict(testpcount)[0]
print("這首詩:", artists[ans])

請輸入要預測的詩:人群 煙火 香檳和氣球 是你 帶我 從派對逃走 逃離 人間 耳語和騷動 這裡 只有 你我和星空 你是 何時 靜靜靠近我 你是 何時 偷偷拯救我 在我 掌心 放了 一顆糖果  人類 為何 都愛看天空 背上 卻都 有一對傷口 生在 這個 凡人的星球 是否 不配 作天使的夢 你將 你的 翅膀給了我 帶我 穿越 狼群和鏡頭 讓我 能夠 品嚐 片刻自由  因為你 所以我 愛上那 片天空 天空下 我在祈求 那是你 牽著我 更多更詳盡歌詞 在 ※ Mojim.com　魔鏡歌詞網 最深刻 的故事 最永恆 的傳說 不過 是你 是我 能夠 平凡生活  因為你 所以我 所以我 不退縮 再不願 這一生中 有遺憾 不反駁 不再聽 別人說 不在乎 誰能懂 只因 世界 再大 不過 你和我 用最小回憶 堆成宇宙  人生 只是 須臾的剎那 人間 只是 天地的夾縫 人們 何苦 要活成修羅 活在 執著 對錯的牢籠 是你 幫我 停下了沙漏 是你 教我 別害怕闖禍 是你 讓我 活得 與眾不同
李白 的機率: 0.00027672158532088855
杜甫 的機率: 3.0423031903687626e-06
白居易 的機率: 0.9997202361114758
這首詩: 白居易
